In [1]:
import googlemaps
import pandas as pd
import os
from dotenv import load_dotenv
from tqdm import tqdm

In [2]:
load_dotenv()
#api = None # enter API key here

True

In [3]:
gmaps = googlemaps.Client(key=os.getenv('api_key'))
#gmaps = googlemaps.Client(api)

In [4]:
def get_lat_lng(address):
    try:
        geocode_result = gmaps.geocode(address)

        if geocode_result and len(geocode_result) > 0:
            location = geocode_result[0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            return None, None
    except Exception as e:
        return None, None

In [13]:
def get_reviews(search_input):
    try:
        place_result = gmaps.find_place(input=search_input, input_type='textquery')
        if place_result and len(place_result['candidates']) > 0:
            place_id = place_result['candidates'][0]['place_id']
            place_details = gmaps.place(place_id=place_id)
            reviews = place_details['result'].get('reviews', [])
            return [(review['author_name'], review['text'], review.get('rating')) for review in reviews]
        else:
            return None
    except Exception as e:
        return None

In [14]:
file_path = '../data/raw_data/kbh_facilities.csv'
output_file_path = '../data/raw_data/kbh_facilities_with_coordinates_and_reviews_raw_v2.csv'

df = pd.read_csv(file_path, encoding='utf-16')

df['lat'] = None
df['lng'] = None

list = []

for index, row in df.iterrows():
    address = str(row['address']).strip() 
    location = str(row['location']).strip()

    # try getting reviews based on location
    reviews = get_reviews(location)
    
    #if no reviews based on location try the address
    if not reviews:
        get_reviews(address)
                
    lat_lng = None
    #look first if we can find lat and lng for the address
    if address:
        lat_lng = get_lat_lng(address)
    # if there is no address or we can't find the coordiantes using address we use the location to find the lat and lng
    if not lat_lng: 
        lat_lng = get_lat_lng(location)
    
    if reviews:
        for review in reviews:
            review_author, review_text, review_rating = review 
            new_row = row.to_dict()
            if lat_lng:
                new_row['lat'] = lat_lng[0]
                new_row['lng'] = lat_lng[1]
            else:
                new_row['lat'] = None
                new_row['lng'] = None
                
            new_row['author'] = review_author  # add author of text
            new_row['review'] = review_text  # add review text
            new_row['rating'] = review_rating  # add review rating
            list.append(new_row)
    else:
        new_row = row.to_dict()
        if lat_lng:
            new_row['lat'] = lat_lng[0]
            new_row['lng'] = lat_lng[1]
        else:
            new_row['lat'] = None
            new_row['lng'] = None
        
        new_row['author'] = None
        new_row['review'] = None
        new_row['rating'] = None
        list.append(new_row)

new_df = pd.DataFrame(list)
new_df = new_df[new_df['review'].notna() & new_df['review'].ne('')]
new_df.to_csv(output_file_path, index=False, encoding='utf-16')

ValueError: too many values to unpack (expected 2)

In [10]:
pd.read_csv('../data/raw_data/kbh_facilities.csv', encoding='utf-16').fillna(value='None')

,type,activity,location,website,gender,age,special,address
0,gym,Styrke- og grundtræning,SOS Motion,http://www.sosmotion.dk/,both,all,None,"Sundhedshus Østerbro, Randersgade 60, 4 sal, 2..."
1,outdoors,Træningspavillion,None,None,both,all,None,"Kvægtorvsgade, 1710 KBH V"
2,outdoors,Kondisti,Valbyparken,None,both,all,None,"Tudsemindevej, 2450 Valby"
3,gym,Nærgymnastik,LOFskolen,https://lofskolen.dk/kurser/motion-og-sundhed/...,both,all,Målrettet personer der har brug for træning me...,"Østerbrogade 240, 2100 København Ø"
4,ball_sports,Floorball for kvinder 65+ år,BK Skjold,https://www.bkskjold.dk/klub/boldklubben-skjol...,women,seniors,None,"Nørrebrogade 208, 2200 Kbh. N"
...,...,...,...,...,...,...,...,...
601,yoga,Yoga,AH Sport,https://www.ahsport.dk/,both,all,None,Støberiet
602,yoga,Yoga,SheZone,https://www.shezone.dk/,women,all,None,"Jagtvej 34, 2200 København N"
603,gym,Motionsgymnastik,Idrætsforeningen Kæmperne,https://ifk98.dk/default.aspx,both,all,None,None
604,ball_sports,Floorball,PACT/Copenhagen Floorball Club,https://pact.dk/motionsfloorball-pact-copenhag...,both,all,None,"Julius Andersens Vej 1, 2500 Valby"


In [11]:
df = pd.read_csv('../data/raw_data/kbh_facilities_with_coordinates_and_reviews_raw_v2.csv', encoding='utf-16').fillna(value='None')
df

,type,activity,location,website,gender,age,special,address,lat,lng,author,review,rating


In [12]:
df.isna().sum()

type        0
activity    0
location    0
website     0
gender      0
age         0
special     0
address     0
lat         0
lng         0
author      0
review      0
rating      0
dtype: int64